In [1]:
%load_ext nb_black
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<IPython.core.display.Javascript object>

In [7]:
import pandas as pd
import numpy as np
import scipy
from fastFM import als
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import GridSearchCV
from source.utils import train_test_split_feature, rmse_scorer

<IPython.core.display.Javascript object>

In [3]:
feature = pd.read_csv("data/feature.csv")

# sampled_users = pd.Series(feature.user_id.unique()).sample(500)
# feature = feature[feature.user_id.isin(sampled_users)]

feature.shape

(1814134, 85)

<IPython.core.display.Javascript object>

In [5]:
cols = [
    "user_id",
    #  'business_id',
    "review_stars",
    "review_date",
    #  'review_props',
    #  'user_review_count',
    "user_elite",
    #  'user_fans',
    "user_average_stars",
    #  'user_compliment',
    #  'user_yelping_years',
    "business_stars",
    "business_review_count",
    "American (New)",
    "American (Traditional)",
    "Arts & Entertainment",
    "Asian Fusion",
    "Bagels",
    "Bakeries",
    "Barbeque",
    "Bars",
    "Beer",
    "Beer Bar",
    "Breakfast & Brunch",
    "Breweries",
    "Buffets",
    "Burgers",
    "Cafes",
    "Caterers",
    "Chicken Wings",
    "Chinese",
    "Cocktail Bars",
    "Coffee & Tea",
    "Comfort Food",
    "Delis",
    "Desserts",
    "Diners",
    "Ethnic Food",
    "Event Planning & Services",
    "Fast Food",
    "Food",
    "Food Delivery Services",
    "French",
    "Gastropubs",
    "Gluten-Free",
    "Greek",
    "Hawaiian",
    "Hot Dogs",
    "Hotels",
    "Hotels & Travel",
    "Ice Cream & Frozen Yogurt",
    "Indian",
    "Italian",
    "Japanese",
    "Juice Bars & Smoothies",
    "Korean",
    "Latin American",
    "Lounges",
    "Mediterranean",
    "Mexican",
    "Middle Eastern",
    "Music Venues",
    "Nightlife",
    "Noodles",
    "Pizza",
    "Pubs",
    "Salad",
    "Sandwiches",
    "Seafood",
    "Soup",
    "Southern",
    "Specialty Food",
    "Sports Bars",
    "Steakhouses",
    "Sushi Bars",
    "Tacos",
    "Tapas/Small Plates",
    "Tex-Mex",
    "Thai",
    "Vegan",
    "Vegetarian",
    "Venues & Event Spaces",
    "Vietnamese",
    "Wine & Spirits",
    "Wine Bars",
]

selected_feature = feature[cols]

X_train, X_test, y_train, y_test = train_test_split_feature(selected_feature.copy())
print(f"X_train: {len(X_train)}, X_test: {len(X_test)}")

X_train = scipy.sparse.csc_matrix(X_train.values)
X_test = scipy.sparse.csc_matrix(X_test.values)

fm = als.FMRegression(n_iter=1000, init_stdev=0.1, rank=20, l2_reg_w=0.1, l2_reg_V=0.5)
fm.fit(X_train, y_train)
y_pred = fm.predict(X_test)

"rmse:", rmse(y_test, y_pred)

X_train: 1578574, X_test: 235560


('rmse:', 1.406416238414059)

<IPython.core.display.Javascript object>

In [ ]:
params = {
    "l2_reg_w": np.linspace(0.01, 0.3, num=3),
    "l2_reg_V": np.linspace(0.1, 0.5, num=3),
}

fm = als.FMRegression(n_iter=1000, init_stdev=0.1, rank=20)
grid = GridSearchCV(fm, params, verbose=3, n_jobs=-1, scoring=rmse_scorer)
grid.fit(X_train, y_train)
y_pred = grid.predict(X_test)

"rmse:", rmse(y_test, y_pred)

/Users/Ikkei/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


In [ ]:
grid.best_params_